In [1]:
#Galay
import random

n = 8
M = []
for i in range(4):
    M += [random.randint(0,1)]
#print(M)

S = []
for i in range(n):
    S += [random.randint(0,1)]
#print(S)

def whitening (message):
    r= round(len(message)/len(S))
    
    W = []
    if r>1:
        for i in range (1,r+1):
            m = message[:len(S)]
            W = W + list(a^b for a,b in zip(m,S))
            message = message[len(S):]
    else :
        W = list(a^b for a,b in zip(message,S))
    
    return W
t = whitening(M)

#print(M)
print(t)

[1, 1, 1, 0]


In [ ]:
#Yassmine
# Hamming code (7,4):

K = 4
def parity(b, indicies):
    # Compute the parity bit for the given string b and indicies
    c = ""
    for i in indicies:
        c += b[i]
    return str(str.count(c, "1") % 2)

def hamming(bits):
    # Return given 4 bits plus parity bits for bits (1,2,3), (2,3,4) and (1,3,4)
    a1 = parity(bits, [0,1,3])
    a2 = parity(bits, [0,2,3])
    a3 = parity(bits, [1,2,3])
    return a1 + a2 + bits[0] + a3 + bits[1:] 

def hamming_encode(b):
    # Read in K=4 bits at a time and write out those plus parity bits
    while len(b) >= K:
        L = b[0:K]
        input(hamming(L))
        b = b[K:]
hamming_encode('0001')

In [ ]:
#Rachid
import numpy as np
def to_nm_list(A, n, m):
    S = [[0 for i in range(m)] for j in range(n)]
    if len(A) == n*m:
        c=0
        for k in range(n):
            for i in range(m):
                S[k][i] = A[c]
                c+=1
    elif len(A) < n*m:
        B = ['0' for i in range(n*m - len(A))]
        b=''
        b=b.join(B)
        A+=b
        S = to_nm_list(A, n, m)
    else:
        S=[]
        k=len(A)//(n*m)
        for i in range(k):
            M=A[i*(n*m):((i+1)*(n*m))]
            S.append(to_nm_list(M, n, m))
        A = A[k*(m*n):]
        if A != '':
            S.append(to_nm_list(A, n, m))
    return (S)
#to_nm_list('111111111', 4, 3)

def interleaving(A, sf, cr):
    O=''
    p=[]
    A=to_nm_list(A, sf, cr+4)
    #print(A)
    if len(A[1]) == 4+cr :
        S = [[0 for i in range(sf)] for j in range(4+cr)]
        for i in range(0, 4+cr):
            for j in range(0, sf):
                k =sf-np.mod((i-j), sf) - 1
                l =cr+3-i
                S[i][j] = A[k][l]
        O=O.join(sum(S, []))
    else:
        for i in A:
            I=''
            #print(i)
            I=I.join(sum(i, []))
            #print(I)
            p.append(interleaving(I, sf, cr))
            #print(p)
        O=O.join(p)
    return(O)
#interleaving('000111111111111111001', 3, 3)

In [ ]:
#Oussama
import numpy as np
from scipy.signal import chirp
import matplotlib.pyplot as plt
from cmath import phase
from cmath import exp,pi                      

#convertion binaire gray en décimal
def gray2dec(num):
    """retourne le nombre entier correspondant au code Gray num"""
    shift = 1
    while True:
        idiv = num >> shift
        num ^= idiv
        if idiv <= 1 or shift == 32: 
            return num
        shift <<= 1
        
        
#génération d'un seul chirp
def mod_LoRa(SF,data,B):
    #constantes
    M=2**SF
    TS=M/(B[1]-B[0])
    l=gray2dec(int(data,2))
    D=np.linspace(0, TS, num=M)
    S=[(i*(B[1]-B[0])/(TS))-(B[1]-B[0])/2 for i in D]
    A=S[0:l+1]
    B=S[l+1:len(S)]
    Ya=[]
    Yb=[]
    Ya=[exp(2j*pi*((1/2*M)*(((B[1]-B[0])*M/TS)**2)*h*h+(-0.5+l/M)*((B[1]-B[0])*M/TS)*h)) for h in range(len(A))]
    Yb=[exp(2j*pi*((1/2*M)*(((B[1]-B[0])*M/TS)**2)*h*h+(-1.5+l/M)*((B[1]-B[0])*M/TS)*h)) for h in range(len(B))]
    X=B+A
    IQ=Yb+Ya
    #plt.plot(D, X)
    #plt.xlabel('t (sec)')
    #plt.show()
    return X,IQ


#génération des chirps du stream de bits avec un préambule de 2 up chirps puis 2 down chirps
def mod_seq(SF,data,B):
    l=str(data)
    M=2**SF
    TS=M/(B[1]-B[0])
    t1,t2=mod_LoRa(SF,'0',B)
    Y=t2*2
    p=t1*2
    Y=Y+[i.conjugate() for i in Y]
    pa=[-i for i in p]
    p=p+pa
    if len(l)%SF==0:
        b=len(l)//SF
        f=[] 
        for i in range(0,b):
            t1,t2=mod_LoRa(SF,l[SF*i:SF*i+SF],B)
            f=f+t1
            Y=Y+t2
        D=np.linspace(0, b*TS, num=len(f)+4*M)
        plt.plot(D, p+f) 
        plt.show()
    else:
        b=int(len(l)/SF)
        f=[]
        for i in range(0,b):
            t1,t2=mod_LoRa(SF,l[SF*i:SF*i+SF],B)
            f=f+t1
            Y=Y+t2
        t1,t2=mod_LoRa(SF,l[b:b+len(l)%SF],B)
        f=f+t1
        Y=Y+t2
        D=np.linspace(0, (b+1)*TS, num=len(f)+4*M)
        plt.plot(D, p+f)
        plt.show()
    return Y 
    

In [ ]:
#Hanane
import numpy
from matplotlib.pyplot import *
import random
import math
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal

def awgn(s,SNRdB):
    
    #AWGN channel
    #Add AWGN noise to input signal. The function adds AWGN noise vector to signal 's' to generate a resulting signal vector 'r' of specified SNR in dB. It also
    #returns the noise vector 'n' that is added to the signal 's' and the power spectral density N0 of noise added
    #Parameters:
       # s : input/transmitted signal vector
        #SNRdB : desired signal to noise ratio (expressed in dB) for the received signal
        #r : received signal vector (r=s+n)

    gamma = 10**(SNRdB/10) #SNR to linear scale
    
    P=sum(np.abs(s)**2)/len(s) #Actual power in the vector
    
    N0=P/gamma # Find the noise spectral density

    if isrealobj(s):# check if input is real/complex object type
       n = sqrt(N0/2)*standard_normal(np.shape(s)) # computed noise
    else:
       n = sqrt(N0/2)*(standard_normal(np.shape(s))+1j*standard_normal(s.shape))


    r = s + n # received signal

    return r

#awgn([1,1,1],SNRdB=-9)

